In [6]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv(r'C:\Users\lyons\OneDrive\Desktop\IML-FALL2025---Voter-Bias\output_data\cfb\master_bias_file_cfb.csv')

In [ ]:
df

,Pollster (v),Season,Week,Rank,Team (t),ap_rank,average,median,"bias1(v, t)","bias2(v, t)","bias3(v, t)"
0,adam-jude,2014,1,1,florida-state-seminoles,1.0,3.067,3.0,0.000000,-2.067000,-2.000000
1,adam-sparks,2014,1,1,florida-state-seminoles,1.0,3.067,3.0,0.000000,-2.067000,-2.000000
2,adam-zucker,2014,1,1,florida-state-seminoles,1.0,3.067,3.0,0.000000,-2.067000,-2.000000
3,bill-rabinowitz,2014,1,1,florida-state-seminoles,1.0,3.067,3.0,0.000000,-2.067000,-2.000000
4,bob-asmussen,2014,1,1,florida-state-seminoles,1.0,3.067,3.0,0.000000,-2.067000,-2.000000
...,...,...,...,...,...,...,...,...,...,...,...
255493,stephen-means,2024,5,25,nebraska-cornhuskers,26.0,21.333,22.0,-0.040000,0.171893,0.136364
255494,steven-johnson,2024,5,25,washington-state-cougars,26.0,23.923,24.0,-0.040000,0.045019,0.041667
255495,tom-murphy,2024,5,25,indiana-hoosiers,26.0,22.875,23.5,-0.040000,0.092896,0.063830
255496,trevor-hass,2024,5,25,indiana-hoosiers,26.0,22.875,23.5,-0.040000,0.092896,0.063830


In [8]:
total_pollsters = (
    df.groupby(["Season", "Week"])["Pollster (v)"]
    .nunique()
    .reset_index(name="total_pollsters")
)
team_votes = (
    df.groupby(["Season", "Week", "Team (t)"])
    .agg(
        n_ranked=("Pollster (v)", "nunique"),
        sum_ranks=("Rank", "sum"),
        ranks_list=("Rank", lambda x: list(x))
    )
    .reset_index()
)
team_votes = team_votes.merge(total_pollsters, on=["Season", "Week"])
team_votes["avg_adj"] = (
    team_votes["sum_ranks"] +
    26 * (team_votes["total_pollsters"] - team_votes["n_ranked"])
) / team_votes["total_pollsters"]

team_votes["median_adj"] = team_votes.apply(
    lambda row: np.median(row["ranks_list"] + [26] * (row["total_pollsters"] - row["n_ranked"])),
    axis=1
)
compare = team_votes[["Season", "Week", "Team (t)", "avg_adj", "median_adj"]]
sample = compare.head(5)

print(sample)


   Season  Week                  Team (t)    avg_adj  median_adj
0    2014     1      alabama-crimson-tide   3.316667         3.0
1    2014     1  arizona-state-sun-devils  20.050000        19.0
2    2014     1             auburn-tigers   6.033333         6.0
3    2014     1              baylor-bears   9.900000        10.0
4    2014     1       boise-state-broncos  25.833333        26.0
